In [1]:
# -*- coding: utf-8 -*-
import random
import copy
import time
import sys
import math
import tkinter #//GUI模块
import threading
from functools import reduce
import numpy as np
from openpyxl import load_workbook

In [2]:
'''
数据载入
----------
载入所有点的横纵坐标以及积分，分别放在三个数组中

'''

#数据载入

# 城市数，蚁群数量
(city_num, ant_num) = (30,60)
#文件地址
book = load_workbook(filename=r"H:\Desktop\定向越野积分赛路线规划问题\代码\数据\五十城市.xlsx")
#工作薄
sheet = book["定向积分赛"]
#初始化三个数组
distance_x= []
distance_y=[]
height=[]
distance=[ [0.0 for col in range(city_num)] for raw in range(city_num)]
score=[]

#从第二行开始
row_num = 2
while row_num <= city_num+1 :
    #将表中第一列的1-100行数据写入data数组中
    distance_x.append(float(sheet.cell(row=row_num, column=1).value))
    distance_y.append(float(sheet.cell(row=row_num, column=2).value))
    height.append(float(sheet.cell(row=row_num, column=3).value))
    score.append(float(sheet.cell(row=row_num, column=4).value))
    row_num = row_num + 1

def All_distance():
    for i in range(city_num):
        for j in range(city_num):
            distance[i][j]=pow((distance_x[i]-distance_x[j]),2)+pow((distance_y[i]-distance_y[j]),2)+pow(height[i]-height[j],2)
            distance[i][j]=pow(distance[i][j],0.5)
            
All_distance()
# print(city_num,len(distance_x))
# print(distance_x)
# print(distance_y)
# print(score,sum(score))
# print(distance)



In [3]:
### 参数
'''
ALPHA:信息启发因子，值越大，则蚂蚁选择之前走过的路径可能性就越大
      ，值越小，则蚁群搜索范围就会减少，容易陷入局部最优
BETA:Beta值越大，蚁群越就容易选择局部较短路径，这时算法收敛速度会
     加快，但是随机性不高，容易得到局部的相对最优
'''
(ALPHA, BETA, RHO, Q) = (1.0,1.0,0.8,100.0)
(yita_1,yita_2)=(0.5,1)

Max_distance=1400
Max_time=45.0
speed=218
#城市距离和信息素
distance_graph = [ [0.0 for col in range(city_num)] for raw in range(city_num)]
pheromone_graph = [ [1.0 for col in range(city_num)] for raw in range(city_num)]

In [4]:
#----------- 蚂蚁 -----------
class Ant(object):
 
    # 初始化
    def __init__(self,ID):
        
        self.ID = ID                 # ID
        self.__clean_data()          # 初始化出生点
 
    # 初始数据
    def __clean_data(self):
    
        self.path = []               # 当前蚂蚁的路径           
        self.total_distance = 0.0    # 当前路径的总距离
        self.total_score=0           #当前路径的总积分
        self.move_count = 0          # 移动次数
        self.current_city = -1       # 当前停留的城市
        self.open_table_city = [True for i in range(city_num)] # 探索城市的状态
        self.total_time=0.0          #当前总时间
        self.speed=250               #初始移动速度
        self.power=100.0             #当前蚂蚁的体力
        
#         city_index = random.randint(0,city_num-1) # 随机初始出生点        
        city_index = 0 # 设置第一个点为起点
        self.current_city = city_index
        self.path.append(city_index)
        self.open_table_city[city_index] = False
        self.move_count = 1               
    
    def cal_prob(self):
        select_citys_prob = [0.0 for i in range(city_num)]  #存储去下个城市的概率
        total_prob = 0.0
        # 获取去下一个城市的概率
        for i in range(city_num):
            if self.open_table_city[i]:
                try :
                    # 计算概率：与信息素浓度成正比，与距离成反比
                    select_citys_prob[i] = pow(pheromone_graph[self.current_city][i], ALPHA) * pow((score[i]), BETA)
                    total_prob += select_citys_prob[i]
                except ZeroDivisionError as e:
                    print ('Ant ID: {ID}, current city: {current}, target city: {target}'.format(ID = self.ID, current = self.current_city, target = i))
                    sys.exit(1)
        return  select_citys_prob, total_prob          
        
    
    def select_city(self,select_citys_prob, total_prob):
        next_city = -1
        if total_prob > 0.0:
            # 产生一个随机概率,0.0-total_prob
            temp_prob = random.uniform(0.0, total_prob)
            for i in range(city_num):
                if self.open_table_city[i]:
                    # 轮次相减
                    temp_prob -= select_citys_prob[i]
                    if temp_prob < 0.0:
                        next_city = i
                        break
        return next_city
    
    # 选择下一个城市
    def __choice_next_city(self):
        next_city = -1
        select_citys_prob, total_prob=self.cal_prob()
        # 轮盘选择城市
        next_city=self.select_city(select_citys_prob, total_prob)
 
        # 未从概率产生，顺序选择一个未访问城市
        # if next_city == -1:
        #     for i in range(city_num):
        #         if self.open_table_city[i]:
        #             next_city = i
        #             break
            
        if (next_city == -1):
            next_city = random.randint(0, city_num - 1)
            while ((self.open_table_city[next_city]) == False):  # if==False,说明已经遍历过了
                next_city = random.randint(0, city_num - 1)
 
        # 返回下一个城市序号
        return next_city
    
    #计算蚂蚁的体力
    def __cal_power(self):
        power_loss_1=distance_graph[self.current_city][next_city]*yita_1
        power_loss_2=0.0
        if height[next_city]-height[self.current_city]>0:
            power_loss_2=(height[next_city]-height[self.current_city])*yita_2
        else:
            power_loss_2=0.0
        self.power-=power_loss_1+power_loss_2
        if self.power<0.0:
            self.power=0.0
    
    
    
    def __cal_speed(self):
        p=self.power
        v=self.speed
        if p==0.0:
            v=speed*0.2
        else:
            if p<25:
                v=speed*0.4
            else:
                if p<50:
                    v=speed*0.6
                else:
                    if p<75:
                        v=speed*0.8
                    else:
                        v=speed
        self.speed=v
        
#         temp_speed=200
#         if self.total_distance<=200:
#             temp_speed=200
#         else:
#             if self.total_distance<=500:
#                 temp_speed=150
#             else:
#                 if self.total_distance<=1000:
#                     temp_speed=100
#                 else:
#                     temp_speed=50
#         self.speed=temp_speed
    
#     #计算总的时间花费
#     def __cal_total_time(self):
#         temp_time=5
#         d=self.total_distance
#         if self.total_distance<=200:
#             temp_time=self.total_distance/200
#         else: 
#             if self.total_distance<=500:
#                 temp_time=1+(self.total_distance-200)/150
#             else:
#                 if self.total_distance<=1000:
#                     temp_time=2.5+(self.total_distance-500)/100
#                 else:
#                     temp_time=7.5+(self.total_distance-1000)/50
#         self.total_time=temp_time 
    
    
    # 计算路径总距离
    def __cal_total_distance(self):
        
        temp_distance = 0.0
 
        for i in range(0, len(self.path)-1):
            start, end = self.path[i], self.path[i+1]
            temp_distance += distance_graph[start][end]
 
          # 回路
#         end = self.path[0]
#         temp_distance += distance_graph[start][end]
        self.total_distance = temp_distance
        
    # 计算路径总积分
    def __cal_total_score(self):
        
        temp_score = 0
 
        for i in range(0, len(self.path)):
            start= self.path[i]
            temp_score += score[start] 
        self.total_score=temp_score
    
    
    # 移动操作
    def __move(self, next_city):
        
        self.path.append(next_city)                     #将该点加入路径表中
        self.open_table_city[next_city] = False         #将该点加入禁忌表中
        self.total_distance += distance_graph[self.current_city][next_city]        #更新总距离
        self.move_count += 1                            #移动次数加一
        self.total_time+=distance_graph[self.current_city][next_city]/self.speed   #计算总时间
        self.__cal_speed()                              #计算速度
        self.power-=distance_graph[self.current_city][next_city]/yita_1+(height[next_city]-height[self.current_city])/yita_2
        self.current_city = next_city                   #更新当前的点

        
        
    def Can_go(self,next_city):
#         def temp_time(temp_distance):
#             temp_time=0.0
#             if 0<=temp_distance<=200:
#                 temp_time=temp_distance/200
#             else:
#                 if temp_distance<=500:
#                     temp_time=1+(temp_distance-200)/150
#                 else:
#                     if temp_distance<=1000:
#                         temp_time=2.5+(temp_distance-500)/100
#                     else:
#                          temp_time=7.5+(temp_distance-1000)/50
            
#             return temp_time
        
        D1=distance[self.current_city][next_city]
        D2=distance[next_city][1]
        temp_distance=self.total_distance+D1+D2
        temp_time=temp_distance/self.speed
#         print(D1,D2,self.total_distance+D1+D2,Max_distance)
        if temp_time>Max_time:
            self.open_table_city[next_city] = False
            return False
        else:
            return True
    
    # 搜索路径
    def search_path(self):
        # 初始化数据
        self.__clean_data()
#         for i in range(city_num):
        while True in self.open_table_city:
#             print('禁忌表：',self.open_table_city,'数量：',self.open_table_city.count(True))
            next_city =  self.__choice_next_city()
#             print('选择的点：',next_city)
#             print('距离比较：',self.total_distance,0.75*float(Max_distance))
            
            if self.Can_go(next_city):
                if next_city==1:
                    if self.open_table_city.count(True)==1:
                        self.__move(next_city)
                    else:
                        next_city =  self.__choice_next_city()
                else:
                    self.__move(next_city)    

        # 计算路径总长度
        self.__cal_total_distance()
        #计算路径总积分
        self.__cal_total_score()

In [5]:
#----------- TSP问题 -----------
        
class TSP(object):
 
    def __init__(self, root, width = 800, height = 800, n = city_num):
 
        # 创建画布
        self.root = root                               
        self.width = width      
        self.height = height
        # 城市数目初始化为city_num
        self.n = n
        # tkinter.Canvas
        self.canvas = tkinter.Canvas(
                root,
                width = self.width,
                height = self.height,
                bg = "#EBEBEB",             # 背景白色 
                xscrollincrement = 1,
                yscrollincrement = 1
            )
        self.canvas.pack(expand = tkinter.YES, fill = tkinter.BOTH)
        self.title("TSP蚁群算法(n:初始化 e:开始搜索 s:停止搜索 q:退出程序)")
        self.__r = 5
        self.__lock = threading.RLock()     # 线程锁
 
        self.__bindEvents()
        self.new()
        
        # 计算城市之间的距离
        for i in range(city_num):
            for j in range(city_num):
                distance_graph[i][j] = pow((distance_x[i] - distance_x[j]), 2) + pow((distance_y[i] - distance_y[j]), 2)
                distance_graph[i][j] = pow(distance_graph[i][j], 0.5)
 
    # 按键响应程序
    def __bindEvents(self):
 
        self.root.bind("q", self.quite)        # 退出程序
        self.root.bind("n", self.new)          # 初始化
        self.root.bind("e", self.search_path)  # 开始搜索
        self.root.bind("s", self.stop)         # 停止搜索
 
    # 更改标题
    def title(self, s):
 
        self.root.title(s)
 
    # 初始化
    def new(self, evt = None):
 
        # 停止线程
        self.__lock.acquire()
        self.__running = False
        self.__lock.release()
 
        self.clear()     # 清除信息 
        self.nodes = []  # 节点坐标
        self.nodes2 = [] # 节点对象
 
        # 初始化城市节点
        for i in range(len(distance_x)):
            # 在画布上随机初始坐标
            x = distance_x[i]/2.5
            y = distance_y[i]/2.5
            h=height[i]
            self.nodes.append((x, y))
            # 生成节点椭圆，半径为self.__r
            node = self.canvas.create_oval(x - self.__r,
                    y - self.__r, x + self.__r, y + self.__r,
                    fill = "#ff0000",      # 填充红色
                    outline = "#000000",   # 轮廓白色
                    tags = "node",
                )
            self.nodes2.append(node)
            # 显示坐标
            self.canvas.create_text(x,y-10,              # 使用create_text方法在坐标（302，77）处绘制文字
                    text = '('+str(x*2.5)+','+str(y*2.5)+','+str(h)+')',    # 所绘制文字的内容
                    fill = 'black'                       # 所绘制文字的颜色为灰色
                )
            
        # 顺序连接城市
        #self.line(range(city_num))
        
        # 初始城市之间的距离和信息素
        for i in range(city_num):
            for j in range(city_num):
                pheromone_graph[i][j] = 1.0
                
        self.ants = [Ant(ID) for ID in range(ant_num)]  # 初始蚁群
        self.best_ant = Ant(-1)                          # 初始最优解
        self.best_ant.total_distance = 1 << 31           # 初始最大距离
        self.best_ant.total_score = 0                    # 初始总积分
        self.iter = 1                                    # 初始化迭代次数 
            
    # 将节点按order顺序连线
    def line(self, order):
        # 删除原线
        self.canvas.delete("line")
        def line2(i1, i2):
            p1, p2 = self.nodes[i1], self.nodes[i2]
            self.canvas.create_line(p1, p2, fill = "#000000", tags = "line")
            return i2
        
        # order[-1]为初始值
        reduce(line2, order, order[0])
 
    # 清除画布
    def clear(self):
        for item in self.canvas.find_all():
            self.canvas.delete(item)
 
    # 退出程序
    def quite(self, evt):
        self.__lock.acquire()
        self.__running = False
        self.__lock.release()
        self.root.destroy()
        print (u"\n程序已退出...")
        sys.exit()
 
    # 停止搜索
    def stop(self, evt):
        self.__lock.acquire()
        self.__running = False
        self.__lock.release()
        
    # 开始搜索
    def search_path(self, evt = None):
     
        # 开启线程
        self.__lock.acquire()
        self.__running = True
        self.__lock.release()
        
        while self.__running:
            # 遍历每一只蚂蚁
            for ant in self.ants:
                # 搜索一条路径
                ant.search_path()
                # 与当前最优蚂蚁比较
                if ant.total_score > self.best_ant.total_score:
                    # 更新最优解
                    self.best_ant = copy.deepcopy(ant)
            # 更新信息素
            self.__update_pheromone_gragh()
#             self.canvas.create_text(400,7000,text =str(self.best_ant.path),fill = 'black')
            print (u"迭代次数：",self.iter,u"最佳路径总距离：",int(self.best_ant.total_distance),u"最佳路径总时间：",int(self.best_ant.total_time),u'最佳路径总积分：',int(self.best_ant.total_score))
            # 连线
            self.line(self.best_ant.path)
            print(self.best_ant.path)

            # 设置标题
            self.title("TSP蚁群算法(n:随机初始 e:开始搜索 s:停止搜索 q:退出程序) 迭代次数: %d 最佳路径总积分：%d" % (self.iter,self.best_ant.total_score))
            # 更新画布
            self.canvas.update()
            self.iter += 1
#             if self.iter>15:
#                 print(self.best_ant.path)
#                 self.stop()

    # 更新信息素
    def __update_pheromone_gragh(self):
 
        # 获取每只蚂蚁在其路径上留下的信息素
        temp_pheromone = [[0.0 for col in range(city_num)] for raw in range(city_num)]
#         for ant in self.ants:
#             for i in range(1,len(self.best_ant.path)):
#                 start, end = ant.path[i-1], ant.path[i]
#                 # 在路径上的每两个相邻城市间留下信息素，与路径总距离反比
#                 temp_pheromone[start][end] +=ant.total_score * Q / (sum(score))
#                 temp_pheromone[end][start] = temp_pheromone[start][end]
                
                
        for i in range(1,len(self.best_ant.path)):
                start, end = self.best_ant.path[i-1], self.best_ant.path[i]
                # 在路径上的每两个相邻城市间留下信息素，与路径总距离反比
                temp_pheromone[start][end] +=self.best_ant.total_score * Q / (sum(score))
                temp_pheromone[end][start] = temp_pheromone[start][end] 
        # 更新所有城市之间的信息素，旧信息素衰减加上新迭代信息素
        for i in range(city_num):
            for j in range(city_num):
                pheromone_graph[i][j] = pheromone_graph[i][j] * RHO + temp_pheromone[i][j]
                if pheromone_graph[i][j]>100:
                    pheromone_graph[i][j]=100
                if pheromone_graph[i][j]<1:
                    pheromone_graph[i][j]=1
#         print('平均信息素：',np.mean(pheromone_graph),'信息素最大值：',max(pheromone_graph),'信息素最小值：',min(pheromone_graph))    #打印信息素平均值
#         print(pheromone_graph)
        
    # 主循环
    def mainloop(self):
        self.root.mainloop()

In [6]:
#----------- 程序的入口处 -----------
                
if __name__ == '__main__':
    TSP(tkinter.Tk()).mainloop()

迭代次数： 1 最佳路径总距离： 3893 最佳路径总时间： 33 最佳路径总积分： 46
[0, 24, 18, 21, 19, 25, 22, 23, 26, 20, 5, 1]
迭代次数： 2 最佳路径总距离： 3893 最佳路径总时间： 33 最佳路径总积分： 46
[0, 24, 18, 21, 19, 25, 22, 23, 26, 20, 5, 1]
迭代次数： 3 最佳路径总距离： 3893 最佳路径总时间： 33 最佳路径总积分： 46
[0, 24, 18, 21, 19, 25, 22, 23, 26, 20, 5, 1]
迭代次数： 4 最佳路径总距离： 3893 最佳路径总时间： 33 最佳路径总积分： 46
[0, 24, 18, 21, 19, 25, 22, 23, 26, 20, 5, 1]
迭代次数： 5 最佳路径总距离： 3883 最佳路径总时间： 38 最佳路径总积分： 50
[0, 20, 26, 23, 22, 25, 19, 21, 18, 24, 15, 29, 1]
迭代次数： 6 最佳路径总距离： 3902 最佳路径总时间： 38 最佳路径总积分： 51
[0, 24, 15, 29, 3, 18, 21, 19, 25, 22, 23, 5, 2, 1]
迭代次数： 7 最佳路径总距离： 3902 最佳路径总时间： 38 最佳路径总积分： 51
[0, 24, 15, 29, 3, 18, 21, 19, 25, 22, 23, 5, 2, 1]
迭代次数： 8 最佳路径总距离： 3902 最佳路径总时间： 38 最佳路径总积分： 51
[0, 24, 15, 29, 3, 18, 21, 19, 25, 22, 23, 5, 2, 1]
迭代次数： 9 最佳路径总距离： 3902 最佳路径总时间： 38 最佳路径总积分： 51
[0, 24, 15, 29, 3, 18, 21, 19, 25, 22, 23, 5, 2, 1]
迭代次数： 10 最佳路径总距离： 3917 最佳路径总时间： 38 最佳路径总积分： 52
[0, 20, 26, 23, 22, 25, 19, 21, 18, 3, 29, 6, 4, 2, 1]
迭代次数： 11 最佳路径总距离： 3917 最佳路径总时间： 38 最佳路径总积